In [13]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

In [14]:
#import data

df = pd.read_csv("data\produkt_tu_stunde_19480101_20211231_05906.txt", delimiter=';')
df.head(2)

,STATIONS_ID,MESS_DATUM,QN_9,TT_TU,RF_TU,eor
0,5906,1948010101,5,-0.3,90.0,eor
1,5906,1948010102,5,-0.3,89.0,eor


In [15]:
#remove unneeded data

df=df.drop(columns=['STATIONS_ID', 'QN_9', 'eor'])
df.head(2)

,MESS_DATUM,TT_TU,RF_TU
0,1948010101,-0.3,90.0
1,1948010102,-0.3,89.0


In [16]:
#rename MESS_DATUM in Date

df = df.rename(columns={"MESS_DATUM": "Date"})

In [17]:
#convert int to date

df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d%H')
df.head(2)

,Date,TT_TU,RF_TU
0,1948-01-01 01:00:00,-0.3,90.0
1,1948-01-01 02:00:00,-0.3,89.0


In [18]:
#remove leap years from dataset

df = df[~((df.Date.dt.month == 2) & (df.Date.dt.day == 29)) ]

Date     1948-01-01 01:00:00
TT_TU                 -999.0
RF_TU                 -999.0
dtype: object

In [19]:
#remove -999 values form dataset

df = df[~(df.TT_TU == -999)]
df = df[~(df.RF_TU == -999)]
df.min()

Date     1948-01-01 01:00:00
TT_TU                  -21.1
RF_TU                    3.0
dtype: object

In [20]:
#create labeled data set for predictions in one year

prediction_hours = (365*24)
df["Prediction_TT"]=df["TT_TU"].shift(-prediction_hours)
df["Prediction_RF"]=df["RF_TU"].shift(-prediction_hours)

In [21]:
#aufteilen der Daten in x und y Werte

X = np.array(df[["TT_TU"]])
X = X[: len(df)-prediction_hours]

In [22]:
#Ertsellen der y Werte

y = np.array(df["Prediction_TT"])
y = y[:-prediction_hours]

In [23]:
#aufteilen der Daten in Trainings und Test Daten

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [24]:
#erstellen einer varaible mit letzten 365*24 Werten aus dem df

prediction_hours_array = np.array(df['TT_TU'])[-prediction_hours:]

In [26]:
#from sklearn.svm import SVR

#Support Vector Regression mit radial basis function

svr_rbf = SVR(kernel="rbf", C=1e3, gamma=1)
svr_rbf.fit(X_train, y_train)

In [ ]:
#Testen des Models
#model_test = svr_rbf.score(X_test, y_test)

#print("Model Score: ", model_test)